In [2]:
#Run this then restart GCB runtime to get updated packages
!pip install --upgrade scikit-learn numpy pandas shap xgboost lightgbm imbalanced-learn matplotlib seaborn bar-chart-race pandas-alive

     |████████████████████████████████| 23.2 MB 2.4 MB/s 
     |████████████████████████████████| 15.7 MB 116 kB/s 
     |████████████████████████████████| 11.3 MB 26.9 MB/s 
     |████████████████████████████████| 371 kB 59.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 173.5 MB 27 kB/s 
     |████████████████████████████████| 2.0 MB 31.5 MB/s 
     |████████████████████████████████| 189 kB 60.7 MB/s 
     |████████████████████████████████| 10.3 MB 21.1 MB/s 
     |████████████████████████████████| 156 kB 45.2 MB/s 
  Created wheel for shap: filename=shap-0.40.0-cp37-cp37m-linux_x86_64.whl size=509255 sha256=ac78299950a4b45f2bab7713352f2a55adc7308b7adbd52d10289effe32a66eb
  Stored in directory: /root/.cache/pip/wheels/ec/35/84/e304841ac4b910bc95fe9a6e5302eb2507b4333728851dcbfb
Successfully built shap
  Attempting uninstall: numpy
    Found existing installatio

In [1]:
#Mount the google drive to download NCAAM Kaggle Files
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# Imports

# General
import pandas as pd, numpy as np, pandas_alive as pa

# Visualizations
import seaborn as sns, matplotlib.pyplot as plt 

# Utilities
import gc as gc, warnings

warnings.filterwarnings('ignore')
gc.enable()

In [3]:
# Load in data
# Files provided via Kaggle

base_dir = '/content/gdrive/MyDrive/Colab Notebooks/ncaa_competition/MDataFiles_Stage1_Spread/'

teams = pd.read_csv(base_dir+'MTeams.csv')
tourney_detailed_results = pd.read_csv(base_dir+'MNCAATourneyDetailedResults.csv')
reg_detailed_results = pd.read_csv(base_dir+'MRegularSeasonDetailedResults.csv')
seeds = pd.read_csv(base_dir+'MNCAATourneySeeds.csv')
seasons = pd.read_csv(base_dir+'MSeasons.csv')
game_cities = pd.read_csv(base_dir+'MGameCities.csv')
cities = pd.read_csv(base_dir+'Cities.csv')
coaches = pd.read_csv(base_dir+'MTeamCoaches.csv')
team_conferences = pd.read_csv(base_dir+'MTeamConferences.csv')
conferences = pd.read_csv(base_dir+'Conferences.csv')
ordinals = pd.read_csv(base_dir+'MMasseyOrdinals.csv')


In [4]:
# Hard code a March Madness game indentifier, as the tournament games are in a separate df than the regular season games
tourney_detailed_results["TournamentStatus"] = "Y"
reg_detailed_results["TournamentStatus"] = "N"

In [5]:
# Concat the regular season with the tourney games
reg_detailed_results = pd.concat([reg_detailed_results,tourney_detailed_results])

In [6]:
#There should be around 94k games listed in the NCAAM dataset since 2003
print(f'Shape: {reg_detailed_results.shape}')
reg_detailed_results.head()
df = reg_detailed_results

Shape: (93947, 35)


In [7]:
# Merge the NCAAM D1 teams with their respective athletic conferences

## TODO: Chain this for efficiency 
conferences_merge = team_conferences.merge(conferences, how='inner' , on = 'ConfAbbrev')
conferences_teams_merge = conferences_merge.merge(teams, how= 'inner', on = 'TeamID')

In [8]:
# Merge the teams and athletic conference data with the coach for the given season 
teams_conferences_coaches_merge = conferences_teams_merge.merge(coaches, how= 'inner', on = ['TeamID','Season'])

#Add a cumulative year count since 1985 to track years of coaching experience in the data set
teams_conferences_coaches_merge['CareerYears'] = teams_conferences_coaches_merge.groupby(['CoachName'])['CoachName'].cumcount()

In [9]:
#Who are the most experienced active coaches?
teams_conferences_coaches_merge.sort_values(by=['CareerYears'])

,Season,TeamID,ConfAbbrev,Description,TeamName,FirstD1Season,LastD1Season,FirstDayNum,LastDayNum,CoachName,CareerYears
0,1985,1102,wac,Western Athletic Conference,Air Force,1985,2021,0,154,reggie_minton,0
6591,1997,1405,mac,Mid-American Conference,Toledo,1985,2021,0,154,stan_joplin,0
1837,1993,1279,sec,Southeastern Conference,Mississippi,1985,2021,0,154,rob_evans,0
6603,2009,1405,mac,Mid-American Conference,Toledo,1985,2021,0,154,gene_cross,0
1831,1987,1279,sec,Southeastern Conference,Mississippi,1985,2021,0,154,ed_murphy,0
...,...,...,...,...,...,...,...,...,...,...,...
10088,2007,1243,big_twelve,Big 12 Conference,Kansas St,1985,2021,0,154,bob_huggins,34
7487,2019,1393,acc,Atlantic Coast Conference,Syracuse,1985,2021,0,154,jim_boeheim,34
9552,2019,1181,acc,Atlantic Coast Conference,Duke,1985,2021,0,154,mike_krzyzewski,34
7488,2020,1393,acc,Atlantic Coast Conference,Syracuse,1985,2021,0,154,jim_boeheim,35


In [10]:
"""
Since the data is formatted in a way that is 
showing the score line as relative to the winner of the game instead of individual team performances, 
the regular season data needs to be split into 3 sections


# 1. Neutral Data
# 2. Winner Data
# 3. Loser Data
# All the stats will be maintained for each individual team performance, but the winner and loser status will be dropped.
# The neutral data will then be added back into both the winner and loser paritions, effectively doubling the size of the data set
"""

'\nSince the data is formatted in a way that is \nshowing the score line as relative to the winner of the game instead of individual team performances, \nthe regular season data needs to be split into 3 sections\n\n\n# 1. Neutral Data\n# 2. Winner Data\n# 3. Loser Data\n# All the stats will be maintained for each individual team performance, but the winner and loser status will be dropped.\n# The neutral data will then be added back into both the winner and loser paritions, effectively doubling the size of the data set\n'

In [11]:
# Create neutral data
neutral_data = reg_detailed_results[['Season','DayNum','NumOT','WLoc','TournamentStatus']]

In [12]:
# Slice the winner data and change all column names, reset the index
winner_data = reg_detailed_results[['Season','DayNum','NumOT','WLoc','WTeamID','LTeamID','WScore','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA','WOR','WDR','WAst','WTO','WStl','WBlk','TournamentStatus']]
winner_data.columns = ('Season','DayNum','NumOT','WLoc','TeamID','OppID','Score','FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','TournamentStatus')
winner_data.reset_index()

#Keep the 'winner' status of the data by hard coding a feature
winner_data['WInd'] = 1
winner_data['LInd'] = 0

In [13]:
#Slice the loser data and change all column names, reset the index
loser_data = reg_detailed_results[['Season','DayNum','NumOT','WLoc','LTeamID','WTeamID','LScore','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA','LOR','LDR','LAst','LTO','LStl','LBlk','TournamentStatus']]
loser_data.columns = ('Season','DayNum','NumOT','WLoc','TeamID','OppID','Score','FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','TournamentStatus')

#Keep the 'winner' status of the data by hard coding a feature
loser_data['WInd'] = 0
loser_data['LInd'] = 1

In [14]:
# Union the data back togther, now there are performances per team per game on individual rows, and reintigrated with the neutral data. This effectively doubles the rows.
all_game_data = pd.concat([winner_data,loser_data], axis=0, ignore_index=True)

In [15]:
# Keep a rolling count of a teams wins an losses as a season progresses. This is to let the model know the record of the team at the time the game was played.
all_game_data = all_game_data.sort_values(['Season','DayNum']).reset_index()
all_game_data['SznWins'] = all_game_data.groupby(['TeamID','Season']).WInd.apply(lambda x : x.shift().fillna(1).cumsum().astype(int))
all_game_data['SznLosses'] = all_game_data.groupby(['TeamID','Season']).LInd.apply(lambda x : x.shift().fillna(1).cumsum().astype(int))


In [16]:
"""
Hard code the RankingDayNumStandard column since certain NCAAM seasons had differing weekly cutoff dates for the new ordinals to be posted. 
This attempt forces the 2021 ordinal weekly ranking days (every week after day 9) onto the older rankings from previous seasons for the sake of simplicity.
Every NCAAM regular season has 128 game days, which extend into the 154th day for the postseason leading up to the NCAAM Championship Game. 
The March Madness games will be lumped into day 128 of the Massey ordinals rankings since the ordinals do not update after the regular season ends.
"""

ordinals['RankingDayNumStandard'] = 0
ordinals.loc[ordinals['RankingDayNum'].between(0, 9, inclusive=True), 'RankingDayNumStandard'] = 9
ordinals.loc[ordinals['RankingDayNum'].between(10, 16, inclusive=True), 'RankingDayNumStandard'] = 16
ordinals.loc[ordinals['RankingDayNum'].between(17, 23, inclusive=True), 'RankingDayNumStandard'] = 23
ordinals.loc[ordinals['RankingDayNum'].between(24, 30, inclusive=True), 'RankingDayNumStandard'] = 30
ordinals.loc[ordinals['RankingDayNum'].between(31, 37, inclusive=True), 'RankingDayNumStandard'] = 37
ordinals.loc[ordinals['RankingDayNum'].between(38, 44, inclusive=True), 'RankingDayNumStandard'] = 44
ordinals.loc[ordinals['RankingDayNum'].between(45, 51, inclusive=True), 'RankingDayNumStandard'] = 51
ordinals.loc[ordinals['RankingDayNum'].between(52, 58, inclusive=True), 'RankingDayNumStandard'] = 58
ordinals.loc[ordinals['RankingDayNum'].between(59, 65, inclusive=True), 'RankingDayNumStandard'] = 65
ordinals.loc[ordinals['RankingDayNum'].between(66, 72, inclusive=True), 'RankingDayNumStandard'] = 72
ordinals.loc[ordinals['RankingDayNum'].between(73, 79, inclusive=True), 'RankingDayNumStandard'] = 79
ordinals.loc[ordinals['RankingDayNum'].between(80, 86, inclusive=True), 'RankingDayNumStandard'] = 86
ordinals.loc[ordinals['RankingDayNum'].between(87, 93, inclusive=True), 'RankingDayNumStandard'] = 93
ordinals.loc[ordinals['RankingDayNum'].between(94, 100, inclusive=True), 'RankingDayNumStandard'] = 100
ordinals.loc[ordinals['RankingDayNum'].between(101, 107, inclusive=True), 'RankingDayNumStandard'] = 107
ordinals.loc[ordinals['RankingDayNum'].between(106, 114, inclusive=True), 'RankingDayNumStandard'] = 114
ordinals.loc[ordinals['RankingDayNum'].between(115, 121, inclusive=True), 'RankingDayNumStandard'] = 121
ordinals.loc[ordinals['RankingDayNum'].between(122, 154, inclusive=True), 'RankingDayNumStandard'] = 128

ordinals = ordinals.drop(columns=['RankingDayNum'])

In [17]:
# Show the average ordinal ranking given to the NCAAM field per ranking day per season
# An aggregate is required since the Massey ordinal system is a collective poll of 20+ basketball power rankings sources. 
avg_ords = ordinals.groupby(['Season','TeamID','RankingDayNumStandard']).agg('mean').reset_index()

In [18]:
#Synthesize the dates in the game data to match with the Massey ordinal rankings using the same methodology as described above.
all_game_data['RankingDayNumStandard'] = 0
all_game_data.loc[all_game_data['DayNum'].between(0, 9, inclusive=True), 'RankingDayNumStandard'] = 9
all_game_data.loc[all_game_data['DayNum'].between(10, 16, inclusive=True), 'RankingDayNumStandard'] = 16
all_game_data.loc[all_game_data['DayNum'].between(17, 23, inclusive=True), 'RankingDayNumStandard'] = 23
all_game_data.loc[all_game_data['DayNum'].between(24, 30, inclusive=True), 'RankingDayNumStandard'] = 30
all_game_data.loc[all_game_data['DayNum'].between(31, 37, inclusive=True), 'RankingDayNumStandard'] = 37
all_game_data.loc[all_game_data['DayNum'].between(38, 44, inclusive=True), 'RankingDayNumStandard'] = 44
all_game_data.loc[all_game_data['DayNum'].between(45, 51, inclusive=True), 'RankingDayNumStandard'] = 51
all_game_data.loc[all_game_data['DayNum'].between(52, 58, inclusive=True), 'RankingDayNumStandard'] = 58
all_game_data.loc[all_game_data['DayNum'].between(59, 65, inclusive=True), 'RankingDayNumStandard'] = 65
all_game_data.loc[all_game_data['DayNum'].between(66, 72, inclusive=True), 'RankingDayNumStandard'] = 72
all_game_data.loc[all_game_data['DayNum'].between(73, 79, inclusive=True), 'RankingDayNumStandard'] = 79
all_game_data.loc[all_game_data['DayNum'].between(80, 86, inclusive=True), 'RankingDayNumStandard'] = 86
all_game_data.loc[all_game_data['DayNum'].between(87, 93, inclusive=True), 'RankingDayNumStandard'] = 93
all_game_data.loc[all_game_data['DayNum'].between(94, 100, inclusive=True), 'RankingDayNumStandard'] = 100
all_game_data.loc[all_game_data['DayNum'].between(101, 107, inclusive=True), 'RankingDayNumStandard'] = 107
all_game_data.loc[all_game_data['DayNum'].between(106, 114, inclusive=True), 'RankingDayNumStandard'] = 114
all_game_data.loc[all_game_data['DayNum'].between(115, 121, inclusive=True), 'RankingDayNumStandard'] = 121
all_game_data.loc[all_game_data['DayNum'].between(122, 154, inclusive=True), 'RankingDayNumStandard'] = 128

In [19]:
#Add the team's avg ordinal rank to the game data to tell the model what their national rank was when the game was played
df = all_game_data.merge(avg_ords, how = 'left', on = (['TeamID','Season','RankingDayNumStandard']))
df = df.dropna(subset=['OrdinalRank'])

In [20]:
#Add the team's avg ordinal rank to the game data to tell the model what their national rank was when the game was played
df = all_game_data.merge(avg_ords, how = 'left', on = (['TeamID','Season','RankingDayNumStandard']))
df = df.dropna(subset=['OrdinalRank'])

# Grab the opponent's avg ordinal ranking a the time of game played.
avg_ords.columns = ['Season','OppID','RankingDayNumStandard','OppOrdinalRank']
df2 = avg_ords.merge(all_game_data[['OppID','Season','RankingDayNumStandard']], how = 'inner', on = (['OppID','Season','RankingDayNumStandard']))
avg_ords.columns = ['Season','TeamID','RankingDayNumStandard','OrdinalRank']

#Add the team's opponent's avg ordinal rank to the game data to tell the model what their national rank was when the game was played. This can help the model to understand the perceived strength of the opposing team.
df = df.merge(df2 ,how = 'left', on = (['OppID','Season','RankingDayNumStandard']))

In [21]:
#Drop dupes
all_game_data = df.drop_duplicates().reset_index()

In [22]:
#Add shooting percentages to the data with simple division
all_game_data['Avg_FG_%'] = all_game_data['FGM'] / all_game_data['FGA']
all_game_data['3P_%'] = all_game_data['FGM3'] / all_game_data['FGA3']
all_game_data['FT_%'] = all_game_data['FTM'] / all_game_data['FTA']

In [23]:
# Merge the game data with the coaching, athletic conference, and team data
all_game_data = teams_conferences_coaches_merge.merge(all_game_data, how= 'inner', on = ['TeamID','Season'])

In [24]:
# Create a cumulative sum of winning games for a given coach to see how successful they are throughout their career. 
# This, in addition to years of experience, should help the model to understand who the most successful coaches are.
all_game_data['CoachCareerWinCount'] = all_game_data.groupby(['CoachName']).WInd.apply(lambda x : x.shift().fillna(1).cumsum().astype(int))
all_game_data['CoachCareerLossCount'] = all_game_data.groupby(['CoachName']).LInd.apply(lambda x : x.shift().fillna(1).cumsum().astype(int))
all_game_data['CoachCareerWinRate'] = all_game_data['CoachCareerWinCount']/(all_game_data['CoachCareerWinCount']+all_game_data['CoachCareerLossCount'])

In [25]:
#List all current features
all_game_data.columns

Index(['Season', 'TeamID', 'ConfAbbrev', 'Description', 'TeamName',
       'FirstD1Season', 'LastD1Season', 'FirstDayNum', 'LastDayNum',
       'CoachName', 'CareerYears', 'level_0', 'index', 'DayNum', 'NumOT',
       'WLoc', 'OppID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'TournamentStatus', 'WInd',
       'LInd', 'SznWins', 'SznLosses', 'RankingDayNumStandard', 'OrdinalRank',
       'OppOrdinalRank', 'Avg_FG_%', '3P_%', 'FT_%', 'CoachCareerWinCount',
       'CoachCareerLossCount', 'CoachCareerWinRate'],
      dtype='object')

In [26]:
#Since the model shouldn't know the stats of the game beforehand, we chose to use rolling 3 and 5 game averages for every critical basketball statistic. 
# We can drop the "current" data about the individual games during model creation.

## TODO: Turn this into a function with parameter n for the # of games to look back

rolling_cols = ['Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'OrdinalRank', 'OppOrdinalRank','Avg_FG_%','3P_%','FT_%','NumOT']

test = all_game_data.copy()
all_game_data.sort_values(by=['Season', 'DayNum'], inplace=True)
for col in rolling_cols:
  all_game_data['TeamRollingSeason3_{}'.format(str(col))] = (all_game_data.groupby(['TeamID','Season'])[col].rolling(3).mean().sort_index(level=1).values)
  all_game_data['TeamRollingSeason5_{}'.format(str(col))] = (all_game_data.groupby(['TeamID','Season'])[col].rolling(5).mean().sort_index(level=1).values)

all_game_data.reset_index(drop=True, inplace=True)

In [27]:
# By taking the winners location (given) and whether the given team won or not, we can determine who was home and who was away when the game took place
conditions = [
    (all_game_data['WLoc'] == 'H') & (all_game_data['WInd'] == 1),
    (all_game_data['WLoc'] == 'A') & (all_game_data['WInd'] == 0),
    (all_game_data['WLoc'] == 'H') & (all_game_data['WInd'] == 0),
    (all_game_data['WLoc'] == 'A') & (all_game_data['WInd'] == 1),
    (all_game_data['WLoc'] == 'N')
    ]

values = ['Home', 'Home', 'Away', 'Away','Neutral']

all_game_data['Location'] = np.select(conditions, values)

In [28]:
# Filter for only after 2010
all_game_data = all_game_data.loc[(all_game_data['Season'] >= 2010)]

In [29]:
# Using the rolling averages for 3 and 5 games, a rough point estimate can be made by multiplying average scoring by what the type of score awards
all_game_data['ProjectedScoreRolling3'] = ((all_game_data['TeamRollingSeason3_FGM']*2) + (all_game_data['TeamRollingSeason3_FGM3']*3) + (all_game_data['TeamRollingSeason3_FTM']))
all_game_data['ProjectedScoreRolling5'] = ((all_game_data['TeamRollingSeason5_FGM']*2) + (all_game_data['TeamRollingSeason5_FGM3']*3) + (all_game_data['TeamRollingSeason5_FTM']))

In [30]:
# Drop the columns that have been determined not useful (low predictive power, data leakage, etc.) in model creation
drop_cols = ['Unnamed: 0', 'level_0', 'index',
             'FGM', 'FGA', 'FGM3', 'FGA3', 
             'FTM', 'FTA', 'OR', 'DR', 
             'Ast', 'TO', 'Stl', 'Blk', 
             'Avg_FG_%','3P_%','FT_%','NumOT','WInd','LInd','CoachName','TournamentStatus','Description','WLoc','LastD1Season','FirstDayNum','LastDayNum','TeamName']

for col in drop_cols:
  try:
    all_game_data.drop(col, inplace=True, axis=1)
  except:
    print("Couldn't drop: {}".format(col))

Couldn't drop: Unnamed: 0


In [31]:
# Write to .csv for EDA and model creation
all_game_data.to_csv('full_data_all_games.csv')
!cp full_data_all_games.csv '/content/gdrive/MyDrive/Colab Notebooks/ncaa_competition/MDataFiles_Stage1_Spread/'